In [ ]:
import os
import rosbag
from cfg.dataset.cfg_sequence import dataset_sequence_calib_used_dict
from data_loader.ros_msg.pointcloud import PointCloud
from data_loader.ros_msg.image import Image
from data_loader.ros_msg.odometry import Odometry
from data_loader.calib.intrinsic_extrinsic_loader import IntrinsicExtrinsicLoader

In [ ]:
def write_alg_bag_to_data(sequence_name, algorithm, rosbag_path, dataset_path, rostopic_msg_frameid_dict):
  ##### Open the rosbag
  input_bag = rosbag.Bag(rosbag_path)
  print('Finish reading bag, start loading messages, and writing message to data folder')
  print('Will save messages to algorithm_result'.format(os.path.join(dataset_path, sequence_name, algorithm)))

  ##### Initialize the lidar object of Sensor class
  # Ouster_undistorted
  print('Loading Ouster_undistorted messages...')
  pointcloud = PointCloud(sensor_type='vlp')
  output_data_path = os.path.join(dataset_path, sequence_name, 'algorithm_result'.format(algorithm), 'ouster00_undistort', 'points')
  num_msg = pointcloud.load_messages_write_to_file(bag=input_bag, output_path=output_data_path, topic=rostopic_msg_frameid_dict['ouster_points_undistorted'][0])
  print('     Saving {} Ouster_undistorted points messages to {}!'.format(num_msg, output_data_path))

  # Odometry
  print('Loading Odometry messages...')
  odometry = Odometry(traj_type='TUM', msg_type='nav_msgs/Odometry')
  output_data_path = os.path.join(dataset_path, sequence_name, 'algorithm_result'.format(algorithm), 'odometry')
  num_msg = odometry.load_messages_write_to_file(bag=input_bag, output_path=output_data_path, topic=rostopic_msg_frameid_dict['odometry'][0])
  print('     Saving {} Odometry messages to {}!'.format(num_msg, output_data_path))
  print('Finishing loading Odometry messages!')

  # Camera Odometry
  if algorithm == 'r3live':
    print('Loading Camera_Odometry messages...')
    odometry = Odometry(traj_type='TUM', msg_type='nav_msgs/Odometry')
    output_data_path = os.path.join(dataset_path, sequence_name, 'algorithm_result'.format(algorithm), 'camera_odometry')
    num_msg = odometry.load_messages_write_to_file(bag=input_bag, output_path=output_data_path, topic=rostopic_msg_frameid_dict['camera_odometry'][0])
    print('     Saving {} Camera_Odometry messages to {}!'.format(num_msg, output_data_path))
    print('Finishing loading Camera_Odometry messages!')

  ##### Close the rosbag
  input_bag.close()

In [ ]:
algorithms = ['fastlio2']
for sequence_name, values in dataset_sequence_calib_used_dict.items():
  if not values[2]:
    continue
  for algorithm in algorithms:
    print('Start processing sequence: {} with algorithm_result: {}'.format(sequence_name, algorithm))

    ##### Set up the rosbag path
    input_dataset_path = '/Rocket_ssd/dataset/data_FusionPortable/evaluation/algorithm_result'
    rosbag_path = os.path.join(input_dataset_path, algorithm, sequence_name + '_{}.bag'.format(algorithm))

    ##### Set up the output data path
    dataset_path = '/Rocket_ssd/dataset/data_FusionPortable/sensor_data/'

    ##### Set up the message topic list for different algorithms
    if algorithm =='r3live':
      from cfg.algorithm.cfg_r3live import algorithm_rostopic_msg_frameid_dict
    elif algorithm =='fastlio2':
      from cfg.algorithm.cfg_fastlio2 import algorithm_rostopic_msg_frameid_dict
    print('Finish loading parameters')

    ##### Main function
    write_alg_bag_to_data(sequence_name, algorithm, rosbag_path, dataset_path, algorithm_rostopic_msg_frameid_dict)